## Flipping coins over the telephone

By: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will be demonstrating a minimal example of the coin fliping protocol demonstrated and explained here https://www.cs.cmu.edu/~mblum/research/pdf/coin

Using default capabilities in python we will assume both the identities of alice and bob to show that both parties will agree on the result of a coin toss. In this notebook we will demonstrate this commitment scheme and how each party would deny the other if either of them violate the designated protocol.

## Imports

In [137]:
import string
import random as r

## Utility functions

These lambda functions are to provide short hand names for common functions we will use throughout this notebook

In [12]:
flip_coin = lambda:r.randint(0,1)
generate_secret = lambda: ''.join(r.choice(string.ascii_lowercase) for i in range(5))

## Classes

Here we will define some high level classes with methods to make the steps in this commitment scheme protocol more obvious

In [ ]:
class Person:
    def __init__(self, output):
        self.secret = generate_secret()

In [130]:
def run_simulation():
    print("-------")
    print("+ Alice-Bob choose random strings")
    alice_secret = generate_secret()
    bob_secret = generate_secret()

    print("+ Bob send secret %s to ----> Alice"%bob_secret)

    alice_flip = flip_coin()
    print("+ Alice flips coin %d"%alice_flip)
    tamper = alice_be_truthful()
    if tamper:
        alice_flip = not alice_flip
        print("- Alice decides to tamper to become %d"%alice_flip)
    else:
        alice_flip = alice_flip
        print("+ Alice decides not to tamper to become %d"%alice_flip)
        
    alice_hash = hash(str([alice_flip, alice_secret, bob_secret]))
    print("+ Alice sends bob hash %s"%alice_hash)
    
    bob_guess = flip_coin()
    print("+ Bob announces guess %d"%bob_guess)
    
    truthful = alice_be_truthful()
    modify_secret = alice_be_truthful()
    if truthful:

        alice_send_to_bob = alice_flip
        
        if modify_secret:
            alice_secret = generate_secret()
            print("- Alice modifies secret to %s"%alice_secret)
            print("- Alice recieves guess and sends truthful flip and untruthful secret -> (%d, %s)"%(alice_send_to_bob, alice_secret))
        else:
            print("+ Alice does not modify secret %s"%alice_secret)
            print("+ Alice recieves guess and sends truthful flip and secret -> (%d, %s)"%(alice_send_to_bob, alice_secret))

    else:
        alice_send_to_bob = not alice_flip
        if modify_secret:
            alice_secret = generate_secret()
            print("- Alice modifies secret to %s"%alice_secret)
            print("- Alice recieves guess and sends non truthful flip and non truthful secret -> (%d, %s)"%(alice_send_to_bob, alice_secret))        
        else:
            print("+ Alice does not modify secret %s"%alice_secret)
            print("- Alice recieves guess and sends non truthful flip and truthful secret -> (%d, %s)"%(alice_send_to_bob, alice_secret))        
        
        
    print("----- Bob verifies if he has won -----")
    bob_guess_verify_hash = hash(str([bob_guess, alice_secret, bob_secret]))
    alice_response_verify_hash = hash(str([alice_send_to_bob, alice_secret, bob_secret]))
    
    if alice_hash == bob_guess_verify_hash:
        print("Bob has definately won! yay! :)")
    else:
        print("Bob has possibly lost?")
        
    if alice_hash == alice_response_verify_hash:
        print("Alice told the truth!")
    else:
        print("Alice is a LIAR >:(")

In [135]:
run_simulation()

-------
+ Alice-Bob choose random strings
+ Bob send secret idqso to ----> Alice
+ Alice flips coin 1
+ Alice decides not to tamper to become 1
+ Alice sends bob hash -9038618295808728200
+ Bob announces guess 0
- Alice modifies secret to zrain
- Alice recieves guess and sends truthful flip and untruthful secret -> (1, zrain)
----- Bob verifies if he has won -----
Bob has possibly lost?
Alice is a LIAR >:(
